#### first pass

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import pandas as pd
import time

In [ ]:
## Prep Physiographic Regions
# https://www.sciencebase.gov/catalog/item/631405bbd34e36012efa304e
# tic = time.time()
physio = gpd.read_file('/nas/cee-water/cjgleason/craig/CONUS_ephemeral_data/other_shapefiles/physio.shp'
                       , engine='pyogrio'
                      )
# print('{:.2f}s to make merged shapefile.'.format(time.time()-tic))

In [ ]:
# Dissolve provinces by division
physio = physio.dissolve(by='DIVISION').reset_index()

In [ ]:
# Set CRS to Web Mercator
physio = physio.to_crs(epsg=3857)

In [ ]:
# Drop all columns besides privince, division, and geometry
physio = physio[['DIVISION', 'PROVINCE', 'geometry']]

In [ ]:
# physio

In [ ]:
# physio.plot(column='DIVISION')

In [ ]:
## Process HUC4 basins

In [ ]:
datapath = '/nas/cee-water/cjgleason/craig/CONUS_ephemeral_data/'

In [ ]:
codes_huc2 = ['01','02','03','04','05','06','07','08','09',
              '10','11','12','13','14','15','16','17','18']

In [ ]:
fieldsF = ['GNIS_ID', 'GNIS_Name', 'LengthKM',  'FlowDir',
           'WBArea_Permanent_Identifier', 'FType', 'FCode',
           'NHDPlusID', 'VPUID', 'geometry']
fieldsVAA = ['NHDPlusID', 'StreamOrde', 'FromNode', 'ToNode',
            'LevelPathI', 'TerminalFl', 'TotDASqKm', 'VPUID']
fieldsEROMMA = ['NHDPlusID', 'QBMA', 'VPUID']

In [ ]:
filepath = '/nas/cee-water/cjgleason/craig/CONUS_ephemeral_data/HUC2_01/NHDPLUS_H_0108_HU4_GDB/NHDPLUS_H_0108_HU4_GDB.gdb'

In [ ]:
flow = gpd.read_file(filename=filepath, layer='NHDFlowline', columns=fieldsF, engine='pyogrio')

In [ ]:
flow.FCode.unique()

In [ ]:
 # VAA
vaa = gpd.read_file(filename=filepath, layer='NHDPlusFlowlineVAA',
                    columns=fieldsVAA, engine='pyogrio')
# Merge on VAA
flow = flow.merge(vaa, on=['NHDPlusID', 'VPUID'])

In [ ]:
# EROMMA
eromma = gpd.read_file(filename=filepath, layer='NHDPlusEROMMA',
                        columns=fieldsEROMMA, engine='pyogrio')
# Merge on EROMMA
flow = flow.merge(eromma, on=['NHDPlusID', 'VPUID'])

In [ ]:
flow = flow.to_crs(epsg=3857)

**Filtering**

In [ ]:
# Keep reaches that are stream types or artificial path
# flow = flow.loc[((flow.FCode == 46000) |
#                 (flow.FCode == 46003) |
#                 (flow.FCode == 46006) |
#                 (flow.FCode == 46007) |
#                 (flow.FCode == 55800))]
flow = flow.loc[(flow.FType == 460) | (flow.FType == 558)]
flow.shape

In [ ]:
# Keep reaches that are not terminal paths
flow = flow.loc[flow.TerminalFl == 0]
flow.shape

In [ ]:
# Keep only reaches with non-zero discharge
flow = flow.loc[flow.QBMA > 0]
flow.shape

In [ ]:
# Keep only reaches with non-zero stream order
flow = flow.loc[flow.StreamOrde > 0]

In [ ]:
# Keep only reaches that are outside of NHDWaterbody


In [ ]:
## Merge on physiographic provinces
test = gpd.sjoin(left_df=flow, right_df=physio, how='left', predicate='within')

In [ ]:
test = test.drop(columns='index_right')

In [ ]:
test

In [ ]:
area = gpd.read_file(filepath, layer='NHDWaterbody', 
                     columns='geometry', engine='pyogrio')

In [ ]:
area = area.to_crs(epsg=3857)

In [ ]:
area

In [ ]:
## Merge on physiographic provinces
test1 = gpd.sjoin(left_df=test, right_df=area, how='inner', predicate='within')

In [ ]:
test1 = test.sjoin(df=area, how='inner', predicate='within')

In [ ]:
lake_ids = test1.loc[:,'NHDPlusID'].to_list()

In [ ]:
test3 = test[~test.NHDPlusID.isin(lake_ids)]

In [ ]:
savepath = ('/nas/cee-water/cjgleason/fiona/HMA_small_rivers/07_08_24_figs/')

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))
test3.plot(ax=ax, color='r')
area.plot(ax=ax)

plt.ylim(5200000,5240000)
plt.xlim(-8060000,-8020000)
# plt.savefig(fname=savepath + 'quabbin_removed.png', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))
test3.plot(ax=ax, color='r')
area.plot(ax=ax)

plt.ylim(5200000,5240000)
plt.xlim(-8100000,-8060000)
plt.savefig(fname=savepath + 'conn_removed.png', bbox_inches='tight')

In [ ]:
test1

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))
test.plot(ax=ax, color='r')
area.plot(ax=ax)

plt.ylim(5200000,5240000)
plt.xlim(-8060000,-8020000)
plt.savefig(fname=savepath + 'quabbin_all.png', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))
test1.plot(ax=ax, color='r')
area.plot(ax=ax)
plt.ylim(5200000,5240000)
plt.xlim(-8060000,-8020000)

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))
test1.plot(ax=ax, color='r')
area.plot(ax=ax)
plt.ylim(5200000,5240000)
plt.xlim(-8100000,-8060000)

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))
test.plot(ax=ax, color='r')
area.plot(ax=ax)
plt.ylim(5200000,5240000)
plt.xlim(-8100000,-8060000)
plt.savefig(fname=savepath + 'conn_all.png', bbox_inches='tight')

In [ ]:
## Merge on physiographic provinces
test2 = gpd.sjoin(left_df=test, right_df=area, how='inner', predicate='intersects')

In [ ]:
test2

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))
test2.plot(ax=ax, color='r')
area.plot(ax=ax)
plt.ylim(5200000,5240000)
plt.xlim(-8060000,-8020000)

In [ ]:
## Calculate widths
# Bankfull widths from Bieber et al. 2015, Table 3
bankfull = pd.read_csv('/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC/data/bieger_2015_bankfull_width.csv')

In [ ]:
bankfull

In [ ]:
test = test.merge(bankfull, on='DIVISION', how='left')
# df1.merge(df2[['item_id', 'Description']], on='item_id', how='inner')

In [ ]:
# Calculate width from cumulative drainage area w/ relationship from Bieger et al. 2015
test['WidthM'] = test.a*test.TotDASqKm**test.b

In [ ]:
test

In [ ]:
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100,
       150, 200, 500, 1000]
test['Bin'] = pd.cut(test['WidthM'], bins)

In [ ]:
test.Bin.value_counts()

In [ ]:
# test.rename(columns={'CalcWidths': 'WidthM'})

In [ ]:
max(test.CalcWidths)

In [ ]:
# flow.WBArea_Permanent_Identifier

In [ ]:
test = flow[(flow.GNIS_Name=='East Branch Swift River') & 
            ~(flow.WBArea_Permanent_Identifier.isnull())]

In [ ]:
test.plot()

In [ ]:
# test[test.NHDPlusID==10000900057062]

In [ ]:
area = gpd.read_file(filepath, layer='NHDWaterbody')

In [ ]:
area = area.to_crs(epsg=3857)

In [ ]:
area.plot()

In [ ]:
# flowt = flow.iloc[0:100, :]

In [ ]:
# test = flow.clip(area)

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))
test.plot(ax=ax, color='r')
area.plot(ax=ax)
plt.ylim(5200000,5240000)
plt.xlim(-8060000,-8020000)

In [ ]:
test1 = test.iloc[0:1000, :]

In [ ]:
area_union = area.geometry.union_all()

In [ ]:
test2 = test1[~test1.within(area_union)]

In [ ]:
test2

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))
test1.plot(ax=ax, color='r')
area.plot(ax=ax)
plt.ylim(5200000,5240000)
plt.xlim(-8060000,-8020000)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
test2 = flow[~flow.geometry.within(area_union)]

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))
test.plot(ax=ax)
# plt.ylim(5300000,5350000)
# plt.xlim(-8100000,-8080000)

#### test prepNHD and chase down topology stuff

In [ ]:
import geopandas as gpd
import os
import pandas as pd
from shapely import is_valid, make_valid

In [ ]:
data_path = '/nas/cee-water/cjgleason/craig/CONUS_ephemeral_data/'

In [ ]:
## Set-up
codes_huc2 = ['01','02','03','04','05','06','07','08','09',
          '10','11','12','13','14','15','16','17','18']
fieldsF = ['GNIS_ID', 'GNIS_Name', 'LengthKM',  'FlowDir',
           'WBArea_Permanent_Identifier', 'FType', 'FCode',
           'NHDPlusID', 'VPUID', 'geometry']
fieldsVAA = ['NHDPlusID', 'StreamOrde', 'FromNode', 'ToNode',
            'LevelPathI', 'TerminalFl', 'TotDASqKm', 'VPUID']
fieldsEROMMA = ['NHDPlusID', 'QBMA', 'VPUID']
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200, 500, 1000]

In [ ]:
is_valid(physio.geometry)

In [ ]:
## Prep Physiographic Regions
# https://www.sciencebase.gov/catalog/item/631405bbd34e36012efa304e
physio = gpd.read_file('/nas/cee-water/cjgleason/craig/CONUS_ephemeral_data/other_shapefiles/physio.shp', engine='pyogrio')
# # Repair broken topology DOESN'T MAKE A DIFFERENCE IN THIS CASE
# physio.geometry = make_valid(physio.geometry)
# Set CRS to Web Mercator
physio = physio.to_crs(epsg=3857)
# Dissolve provinces by division
division = physio.dissolve(by='DIVISION').reset_index()
# Drop all columns besides division and geometry
division = division[['DIVISION', 'geometry']]

In [ ]:
is_valid(physio.geometry)

In [ ]:
division.plot()

plt.xlim(-8200000,-7800000)
plt.ylim(5000000,5700000)

In [ ]:
neu = physio[physio.SECTION == 'NEW ENGLAND UPLAND']

In [ ]:
gm = physio[physio.SECTION == 'GREEN MOUNTAIN']

In [ ]:
t = physio[physio.SECTION == 'TACONIC']

In [ ]:
c = physio[physio.SECTION == 'CHAMPLAIN']

In [ ]:
## Get bankfull width coefficients from Bieber et al. 2015, Table 3
bankfull = pd.read_csv('/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC/data/bieger_2015_bankfull_width.csv')  

In [ ]:
## Loop through HUC4 basins, prep the data, and write out new file
# for i in codes_huc2:
i = '01'
        
# Get all HUC4 paths for current HUC2 (excluding WBD)
sub_paths = [fn for fn in os.listdir(os.path.join(data_path, 'HUC2_' + i)) if fn.startswith('NHD')]

In [ ]:
sub_paths

In [ ]:
# for j in sub_paths:
j = sub_paths[7]
j

In [ ]:
file_path = os.path.join(data_path, 'HUC2_' + i,
                j, j + '.gdb')

In [ ]:
## Merging
# Read in NHD flowlines
basin = gpd.read_file(filename=file_path, layer='NHDFlowline',
                      # columns=fieldsF, 
                      engine='pyogrio')
# Set CRS to Pseudo-Mercator https://epsg.io/3857
basin = basin.to_crs(epsg=3857)

In [ ]:
# basin

In [ ]:
# basin.plot()

In [ ]:
# Read in VAA
vaa = gpd.read_file(filename=file_path, layer='NHDPlusFlowlineVAA',
                    columns=fieldsVAA, engine='pyogrio')
# Merge on VAA
basin = basin.merge(vaa, on=['NHDPlusID', 'VPUID'])

In [ ]:
# Read in EROMMA
eromma = gpd.read_file(filename=file_path, layer='NHDPlusEROMMA',
            columns=fieldsEROMMA, engine='pyogrio')
# Merge on EROMMA
basin = basin.merge(eromma, on=['NHDPlusID', 'VPUID'])
# Set CRS to Pseudo-Mercator https://epsg.io/3857
basin = basin.to_crs(epsg=3857)

In [ ]:
## Filtering
# Read in NHD Waterbody polygons
area = gpd.read_file(filename=file_path, layer='NHDWaterbody',
                     columns=['NHDPlusID', 'geometry'], engine='pyogrio')
# Set CRS to Pseudo-Mercator https://epsg.io/3857
area = area.to_crs(epsg=3857)

In [ ]:
# Find all flowlines within waterbodies
# DOES THIS NEED TO BE INTERSECTS OR OTHER
# https://shapely.readthedocs.io/en/stable/manual.html#predicates-and-relationships
subset = basin.sjoin(df=area, how='inner', predicate='within')
# Get IDs of these flowlines
ids = subset.NHDPlusID_left.to_list()

In [ ]:
# Keep only flowlines NOT within waterbodies
basin = basin[~basin.NHDPlusID.isin(ids)]

In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx 

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))
basin.plot(ax=ax, color='r')
area.plot(ax=ax)

plt.ylim(5200000,5240000)
plt.xlim(-8060000,-8020000)
# plt.savefig(fname=savepath + 'quabbin_removed.png', bbox_inches='tight')

In [ ]:
# Keep only reaches that are stream types or artificial path
basin = basin.loc[(basin.FType == 460) | (basin.FType == 558)]
# Keep only reaches that are not terminal paths
basin = basin.loc[basin.TerminalFl == 0]
# Keep only reaches with non-zero discharge
basin = basin.loc[basin.QBMA > 0]
# Keep only reaches with non-zero stream order
basin = basin.loc[basin.StreamOrde > 0]

In [ ]:
basin

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))
basin.plot(ax=ax, color='r')
area.plot(ax=ax)

plt.ylim(5200000,5240000)
plt.xlim(-8060000,-8020000)
# plt.savefig(fname=savepath + 'quabbin_removed.png', bbox_inches='tight')

In [ ]:
## Find the physiographic division each reach is within
basin = basin.sjoin(df=division, how='left',
                predicate='intersect').drop(columns='index_right')

In [ ]:
## Get bankfull widths
# Merge on bankfull width coefficient
basin = basin.merge(bankfull, on='DIVISION', how='left')

In [ ]:
test.columns

In [ ]:
missing = test[test.DIVISION.isnull()].NHDPlusID.to_list()

In [ ]:
len(missing)

In [ ]:
osm = 'https://b.tiles.maps.eox.at/wmts/1.0.0/osm/default/WGS84/8/51/279.jpg'

In [ ]:
savepath = ('/nas/cee-water/cjgleason/fiona/HMA_small_rivers/07_08_24_figs/')

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
test[test.DIVISION.isnull()].plot(ax=ax, color='k')

plt.xlim(-8200000,-7800000)
plt.ylim(5000000,5700000)

neu.plot(ax=ax, alpha=0.5)
gm.plot(ax=ax, alpha=0.5, color='green')
t.plot(ax=ax, alpha=0.5, color='yellow')
c.plot(ax=ax, alpha=0.5, color='orange')

# plt.savefig(fname=savepath + 'NHDPLUS_H_0101_HU4_missing_div.png', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
basin.plot(ax=ax, color='k')

plt.xlim(-8200000,-7800000)
plt.ylim(5000000,5700000)

neu.plot(ax=ax, alpha=0.5)
gm.plot(ax=ax, alpha=0.5, color='green')
t.plot(ax=ax, alpha=0.5, color='yellow')
c.plot(ax=ax, alpha=0.5, color='orange')

plt.savefig(fname=savepath + 'NHDPLUS_H_0101_HU4_div.png', bbox_inches='tight')

In [ ]:
# Calculate width from cumulative drainage area
basin['WidthM'] = basin.a*basin.TotDASqKm**basin.b

In [ ]:
# Plot widths
fig, ax = plt.subplots(figsize=(16,16))
basin.plot(column='WidthM', cmap='coolwarm', ax=ax,
         # legend=True, legend_kwds={"label": "Width (m)", 'shrink': 0.76}
             )

ax.set_facecolor("black")

#### final check for timing and data

In [1]:
import geopandas as gpd
import os
import pandas as pd

In [2]:
## Set-up

In [3]:
data_path = '/nas/cee-water/cjgleason/craig/CONUS_ephemeral_data/'

In [4]:
mdata_path = '/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC/data/'

In [5]:
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200, 500, 1000]

In [6]:
# Define dtypes for lookup tables to preserve leading zeros
dtype_dic= {'HUC4': str, 'HUC2': str, 'toBasin': str, 'level': str}
# Read in HUC lookup table
lookup = pd.read_csv('/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC/data/HUC4_lookup.csv',
                     dtype=dtype_dic)

In [7]:
# Get slurm job index
# i = jobarray index: {slurm}
i = 114;
# Get current HUC2 and HUC4 IDs
hu2 = 'HUC2_' + lookup.loc[i,'HUC2']
hu4 = 'NHDPLUS_H_' + lookup.loc[i,'HUC4'] + '_HU4_GDB'

In [8]:
# Set data filepath
file_path = os.path.join(data_path, hu2, hu4, hu4 + '.gdb')
# Set write filepath
save_path = os.path.join('../narrow_rivers_PIXC_data/prepped_NHD/', hu2)
save_file = hu4 + '_prepped.gpkg'

In [9]:
## Prep Physiographic Regions
# https://www.sciencebase.gov/catalog/item/631405bbd34e36012efa304e
physio = gpd.read_file(filename=os.path.join(data_path, 'other_shapefiles/physio.shp'), engine='pyogrio')
# # Repair broken topology DOESN'T MAKE A DIFFERENCE IN THIS CASE
# physio.geometry = make_valid(physio.geometry)
# Set CRS to Web Mercator
physio = physio.to_crs(epsg=3857)
# Dissolve provinces by division
physio = physio.dissolve(by='DIVISION').reset_index()
# Drop all columns besides division and geometry
physio = physio[['DIVISION', 'geometry']]

ERROR 1: PROJ: proj_create_from_database: Open of /work/pi_cjgleason_umass_edu/.conda/envs/small-rivers-1/share/proj failed


In [10]:
## Get bankfull width coefficients from Bieber et al. 2015, Table 3
bankfull = pd.read_csv(os.path.join(mdata_path, 'bieger_2015_bankfull_width.csv'))

In [11]:
# ## Loop through HUC4 basins, prep the data, and write out new file
# # for i in codes_huc2:
# i = '01'
        
# # Get all HUC4 paths for current HUC2 (excluding WBD)
# sub_paths = [fn for fn in os.listdir(os.path.join(data_path, 'HUC2_' + i)) if fn.startswith('NHD')]

In [12]:
# sub_paths

In [13]:
# # for j in sub_paths:
# j = sub_paths[7]
# j

In [14]:
## Merging
# Read in NHD flowlines
basin = gpd.read_file(filename=file_path, layer='NHDFlowline', engine='pyogrio')
# Set CRS to Pseudo-Mercator https://epsg.io/3857
basin = basin.to_crs(epsg=3857)

/work/pi_cjgleason_umass_edu/.conda/envs/small-rivers-1/lib/python3.12/site-packages/pyogrio/raw.py:196: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiLineString' is converted to 'MultiLineString Z'
  return ogr_read(


In [15]:
# Read in VAA
vaa = gpd.read_file(filename=file_path, layer='NHDPlusFlowlineVAA', engine='pyogrio')
# Merge on VAA
basin = basin.merge(vaa, on=['NHDPlusID', 'VPUID', 'ReachCode'])

In [16]:
# Read in EROMMA
eromma = gpd.read_file(filename=file_path, layer='NHDPlusEROMMA', engine='pyogrio')
# Merge on EROMMA
basin = basin.merge(eromma, on=['NHDPlusID', 'VPUID'])

In [17]:
## Filtering
# Read in NHD Waterbody polygons
area = gpd.read_file(filename=file_path, layer='NHDWaterbody',
                     columns=['NHDPlusID', 'geometry'], engine='pyogrio')
# Set CRS to Pseudo-Mercator https://epsg.io/3857
area = area.to_crs(epsg=3857)

In [18]:
# fig, ax = plt.subplots(figsize=(16,16))
# basin.plot(ax=ax, color='r')
# area.plot(ax=ax)

In [19]:
# fig, ax = plt.subplots(figsize=(16,16))
# basin.plot(ax=ax, color='r')
# area.plot(ax=ax)

# plt.ylim(5900000,6000000)
# plt.xlim(-13600000,-13500000)

In [20]:
# Find all flowlines within waterbodies
# DOES THIS NEED TO BE INTERSECTS--NOPE
# https://shapely.readthedocs.io/en/stable/manual.html#predicates-and-relationships
subset = basin.sjoin(df=area, how='inner', predicate='within')
# # Get IDs of these flowlines
ids = subset.NHDPlusID_left.to_list()

In [21]:
# Keep only flowlines NOT within waterbodies
basin = basin[~basin.NHDPlusID.isin(ids)]

In [22]:
# Keep only reaches that are stream types or artificial path
basin = basin.loc[(basin.FType == 460) | (basin.FType == 558)]
# Keep only reaches that are not terminal paths
basin = basin.loc[basin.TerminalFl == 0]
# Keep only reaches with non-zero discharge
basin = basin.loc[basin.QBMA > 0]
# Keep only reaches with non-zero stream order
basin = basin.loc[basin.StreamOrde > 0]

In [23]:
# fig, ax = plt.subplots(figsize=(16,16))
# basin.plot(ax=ax, color='r')
# area.plot(ax=ax)

# plt.ylim(5900000,6000000)
# plt.xlim(-13600000,-13500000)

In [24]:
## Find the physiographic division each reach is located in
# Using intersects as the province topology is a bit broken
# and I can't mend it fully with shapely or sf
basin = basin.sjoin(df=physio, how='left',
                    predicate='intersects').drop(columns='index_right')

In [25]:
# Drop all reaches where DIVISION == NaN (in Canada and those off the coast)
basin = basin[~basin.DIVISION.isnull()]

In [26]:
## Get bankfull widths
# Merge on bankfull width coefficient
basin = basin.merge(bankfull, on='DIVISION', how='left')

In [27]:
# Calculate width from cumulative drainage area
basin['WidthM'] = basin.a*basin.TotDASqKm**basin.b

In [28]:
## Write out gdf as gpkg file
if not os.path.isdir(save_path):
    os.makedirs(save_path)
basin.to_file(os.path.join(save_path, save_file), driver='GPKG')